# Quantitative Momentum Strategy for S&P500

Building an investing strategy that selects the 50 stocks with the highest price momentum and recommends trades for an equal-weight portfolio of these 50 stocks.

## Basic Strategy

We'll simply look at the one year returns of each stock and recommend certain shares of the ones that have displayed the most promising results

### Library Imports

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
from scipy.stats import percentileofscore as score

### Importing Our List of Stocks

In [2]:
from secrets import IEX_CLOUD_API_TOKEN
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
496,YUM
497,ZBH
498,ZBRA
499,ZION


### Checking returned data from different API Calls

In [3]:
symbol = "AAPL"
url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2739782880481,
 'week52high': 190.2,
 'week52low': 129.6,
 'week52highSplitAdjustOnly': 188.89,
 'week52lowSplitAdjustOnly': 129.64,
 'week52change': 0.14325002478972643,
 'sharesOutstanding': 16659881535,
 'float': 0,
 'avg10Volume': 72803314,
 'avg30Volume': 71529827,
 'day200MovingAvg': 157.57,
 'day50MovingAvg': 155.9,
 'employees': 149096,
 'ttmEPS': 6.18,
 'ttmDividendRate': 0.9227538437282381,
 'dividendYield': 0.005437461780250802,
 'nextDividendDate': '',
 'exDividendDate': '2022-07-24',
 'nextEarningsDate': '2022-10-21',
 'peRatio': 26.8787752915174,
 'beta': 1.2777223839535046,
 'maxChangePercent': 64.78832697291016,
 'year5ChangePercent': 3.6245442210318695,
 'year2ChangePercent': 0.5421890435189949,
 'year1ChangePercent': 0.1365775182554806,
 'ytdChangePercent': -0.06622272670883447,
 'month6ChangePercent': -0.03899636008009946,
 'month3ChangePercent': 0.060144927724437136,
 'month1ChangePercent': 0.17105189527142334,
 'day30Chang

In [4]:
url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/price?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(url).json()
data

168.73

### Executing Batch API Calls & Populating Our DataFrame

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def breakdown(arr, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(arr), n):
        yield arr[i:i + n]   
        
symbol_groups = list(breakdown(stocks['Ticker'], 100))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS',
 'GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
cols = ['Ticker', 'Stock Price', 'One-Year Price Return', 'No. of Shares to Buy']
updated_stocks = pd.DataFrame(columns=cols)

for string in symbol_strings:
    url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(url).json()
    for stock in string.split(','):
        updated_stocks = updated_stocks.append(
        pd.Series([
            stock,
            data[stock]["price"],
            data[stock]["stats"]["year1ChangePercent"],
            "N/A"
        ], index=cols), ignore_index=True)
    
updated_stocks

,Ticker,Stock Price,One-Year Price Return,No. of Shares to Buy
0,A,136.47,-0.134065,N/A
1,AAL,15.20,-0.301616,N/A
2,AAP,197.57,-0.040583,N/A
3,AAPL,165.44,0.134755,N/A
4,ABBV,144.39,0.259523,N/A
...,...,...,...,...
496,YUM,120.32,-0.09908,N/A
497,ZBH,113.72,-0.246659,N/A
498,ZBRA,332.90,-0.426204,N/A
499,ZION,56.00,0.017136,N/A


### Removing Low-Momentum Stocks


In [7]:
updated_stocks.sort_values(by="One-Year Price Return", ascending=False, inplace=True)
updated_stocks = updated_stocks[:50]
updated_stocks.reset_index(inplace=True, drop=True)
updated_stocks

,Ticker,Stock Price,One-Year Price Return,No. of Shares to Buy
0,LB,81.70,2.370031,N/A
1,DVN,58.26,1.329765,N/A
2,OXY,59.26,1.288934,N/A
3,CF,103.52,1.272703,N/A
4,MRO,22.37,0.875406,N/A
5,APA,32.35,0.799348,N/A
6,MCK,350.52,0.761041,N/A
7,COP,96.47,0.726262,N/A
8,MOS,54.04,0.718992,N/A
9,HRB,40.87,0.686257,N/A


### Calculating the Number of Shares to Buy

In [8]:
def calculate_shares(df):
    check = False
    val = input("Enter the value of your investment portfolio here please (as a number): ")
    
    while check is False:
        try:
            val = float(val)
            check = True
        except ValueError:
            print("Please enter a numeric value!")
            val = input("Enter the value of your investment portfolio here please (as a number): ")

    position_size = val / len(df.index)

    print(f"Given your portfolio value at ${val}, you're position size is {position_size}")

    for i in range(len(df.index)):
        df.loc[i, "No. of Shares to Buy"] = position_size // df.loc[i, "Stock Price"]

    print(df)

In [22]:
calculate_shares(updated_stocks)

   Ticker  Stock Price One-Year Price Return No. of Shares to Buy
0      LB        81.70              2.370031             244798.0
1     DVN        58.26              1.329765             343288.0
2     OXY        59.26              1.288934             337495.0
3      CF       103.52              1.272703             193199.0
4     MRO        22.37              0.875406             894054.0
5     APA        32.35              0.799348             618238.0
6     MCK       350.52              0.761041              57058.0
7     COP        96.47              0.726262             207318.0
8     MOS        54.04              0.718992             370096.0
9     HRB        40.87              0.686257             489356.0
10    VLO       108.92              0.676492             183621.0
11   DLTR       173.15               0.67391             115506.0
12    MPC        90.61              0.657135             220726.0
13    XOM        88.56              0.653163             225835.0
14    EOG 

c:\Python310\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## Improved Strategy

Earlier, we had just recommended stocks based on 1 year returns. However, in the real world, momentum investing is done based on "high-quality" momentum stocks and "low-quality" momentum stocks. The difference is that high-quality stocks have shown a positive trend for a long period of time whereas low-quality stocks have demonstrated just an upward spike in performance but don't seem to bear any noticeable profits prior to that. 

Now, we'll recommend high-quality momentum stocks by taking into account their returns over 1 month, 3 months, 6 months, and 1 year, and checking the trend across these time frames.

In [10]:
hqm_cols = ["Ticker", "Stock Price", "No. of Shares to Buy", "HQM Score", "1-Year Price Return", 
            "1-Year Return Percentile", "6-Month Price Return", "6-Month Return Percentile", 
            "3-Month Price Return", "3-Month Return Percentile", "1-Month Price Return", "1-Month Return Percentile"]

hqm_stocks = pd.DataFrame(columns=hqm_cols)
hqm_stocks

for string in symbol_strings:
    url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(url).json()
    for stock in string.split(','):
        hqm_stocks = hqm_stocks.append(
            pd.Series(
                [
                    stock,
                    data[stock]["price"],
                    "N/A",
                    "N/A",
                    data[stock]["stats"]["year1ChangePercent"],
                    "N/A",
                    data[stock]["stats"]["month6ChangePercent"],
                    "N/A",
                    data[stock]["stats"]["month3ChangePercent"],
                    "N/A",
                    data[stock]["stats"]["month1ChangePercent"],
                    "N/A"
                ], index=hqm_cols), ignore_index=True)
        
hqm_stocks

,Ticker,Stock Price,No. of Shares to Buy,HQM Score,1-Year Price Return,1-Year Return Percentile,6-Month Price Return,6-Month Return Percentile,3-Month Price Return,3-Month Return Percentile,1-Month Price Return,1-Month Return Percentile
0,A,138.90,N/A,N/A,-0.134259,N/A,-0.051111,N/A,0.095941,N/A,0.109133,N/A
1,AAL,15.33,N/A,N/A,-0.309144,N/A,-0.107281,N/A,-0.20861,N/A,0.064736,N/A
2,AAP,193.57,N/A,N/A,-0.039011,N/A,-0.140041,N/A,-0.032067,N/A,0.063666,N/A
3,AAPL,166.51,N/A,N/A,0.138445,N/A,-0.038657,N/A,0.059947,N/A,0.177947,N/A
4,ABBV,138.09,N/A,N/A,0.260614,N/A,-0.000981,N/A,-0.088261,N/A,-0.097226,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,120.49,N/A,N/A,-0.10204,N/A,-0.046444,N/A,0.029872,N/A,0.009572,N/A
497,ZBH,116.53,N/A,N/A,-0.247169,N/A,-0.074629,N/A,-0.074485,N/A,0.060769,N/A
498,ZBRA,339.70,N/A,N/A,-0.423352,N/A,-0.344215,N/A,-0.047218,N/A,0.100389,N/A
499,ZION,56.00,N/A,N/A,0.017355,N/A,-0.241682,N/A,-0.061984,N/A,0.05817,N/A


### Calculating Momentum Percentiles

In [14]:
periods = ["1-Year", "6-Month", "3-Month", "1-Month"]

for row in hqm_stocks.index:
    for period in periods:
        percentile_col = f"{period} Return Percentile"
        return_col = f"{period} Price Return"
        
        #error-proofing
        if hqm_stocks.loc[row, return_col] == None:
            hqm_stocks.loc[row, return_col] = 0
            
        hqm_stocks.loc[row, percentile_col] = score(hqm_stocks[return_col], hqm_stocks.loc[row, return_col]) / 100
        
hqm_stocks

,Ticker,Stock Price,No. of Shares to Buy,HQM Score,1-Year Price Return,1-Year Return Percentile,6-Month Price Return,6-Month Return Percentile,3-Month Price Return,3-Month Return Percentile,1-Month Price Return,1-Month Return Percentile
0,A,138.90,N/A,N/A,-0.134259,0.339321,-0.051111,0.528942,0.095941,0.854291,0.109133,0.692615
1,AAL,15.33,N/A,N/A,-0.309144,0.10978,-0.107281,0.37525,-0.20861,0.065868,0.064736,0.483034
2,AAP,193.57,N/A,N/A,-0.039011,0.500998,-0.140041,0.289421,-0.032067,0.393214,0.063666,0.47505
3,AAPL,166.51,N/A,N/A,0.138445,0.790419,-0.038657,0.560878,0.059947,0.762475,0.177947,0.89022
4,ABBV,138.09,N/A,N/A,0.260614,0.892216,-0.000981,0.642715,-0.088261,0.243513,-0.097226,0.023952
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,120.49,N/A,N/A,-0.10204,0.403194,-0.046444,0.538922,0.029872,0.668663,0.009572,0.215569
497,ZBH,116.53,N/A,N/A,-0.247169,0.161677,-0.074629,0.473054,-0.074485,0.275449,0.060769,0.457086
498,ZBRA,339.70,N/A,N/A,-0.423352,0.037924,-0.344215,0.031936,-0.047218,0.355289,0.100389,0.664671
499,ZION,56.00,N/A,N/A,0.017355,0.628743,-0.241682,0.103792,-0.061984,0.307385,0.05817,0.451098


### Calculating the High Quality Momentum Score

In [15]:
from statistics import mean

for row in hqm_stocks.index:
    percentiles = []
    for period in periods:
        percentiles.append(hqm_stocks.loc[row, f"{period} Return Percentile"])
    hqm_stocks.loc[row, "HQM Score"] = mean(percentiles)
    
hqm_stocks

,Ticker,Stock Price,No. of Shares to Buy,HQM Score,1-Year Price Return,1-Year Return Percentile,6-Month Price Return,6-Month Return Percentile,3-Month Price Return,3-Month Return Percentile,1-Month Price Return,1-Month Return Percentile
0,A,138.90,N/A,0.603792,-0.134259,0.339321,-0.051111,0.528942,0.095941,0.854291,0.109133,0.692615
1,AAL,15.33,N/A,0.258483,-0.309144,0.10978,-0.107281,0.37525,-0.20861,0.065868,0.064736,0.483034
2,AAP,193.57,N/A,0.414671,-0.039011,0.500998,-0.140041,0.289421,-0.032067,0.393214,0.063666,0.47505
3,AAPL,166.51,N/A,0.750998,0.138445,0.790419,-0.038657,0.560878,0.059947,0.762475,0.177947,0.89022
4,ABBV,138.09,N/A,0.450599,0.260614,0.892216,-0.000981,0.642715,-0.088261,0.243513,-0.097226,0.023952
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,120.49,N/A,0.456587,-0.10204,0.403194,-0.046444,0.538922,0.029872,0.668663,0.009572,0.215569
497,ZBH,116.53,N/A,0.341816,-0.247169,0.161677,-0.074629,0.473054,-0.074485,0.275449,0.060769,0.457086
498,ZBRA,339.70,N/A,0.272455,-0.423352,0.037924,-0.344215,0.031936,-0.047218,0.355289,0.100389,0.664671
499,ZION,56.00,N/A,0.372754,0.017355,0.628743,-0.241682,0.103792,-0.061984,0.307385,0.05817,0.451098


### Removing Low-Momentum Stocks

In [16]:
hqm_stocks.sort_values(by="HQM Score", ascending=False, inplace=True)
hqm_stocks = hqm_stocks[:50]
hqm_stocks.reset_index(inplace=True, drop=True)
hqm_stocks

,Ticker,Stock Price,No. of Shares to Buy,HQM Score,1-Year Price Return,1-Year Return Percentile,6-Month Price Return,6-Month Return Percentile,3-Month Price Return,3-Month Return Percentile,1-Month Price Return,1-Month Return Percentile
0,COG,23.16,N/A,0.95509,0.353617,0.928144,0.188138,0.934132,0.263359,0.988024,0.253421,0.97006
1,SNPS,385.99,N/A,0.9501,0.288718,0.904192,0.21763,0.9501,0.34051,0.992016,0.239775,0.954092
2,CDNS,187.93,N/A,0.9501,0.237205,0.876248,0.276834,0.978044,0.252244,0.986028,0.240278,0.96008
3,AIV,8.65,N/A,0.939122,0.205268,0.856287,0.228322,0.954092,0.386675,0.998004,0.226663,0.948104
4,GWW,561.03,N/A,0.929142,0.299526,0.91018,0.155299,0.906188,0.160274,0.948104,0.230204,0.952096
5,UNM,36.00,N/A,0.919162,0.478379,0.958084,0.363413,0.992016,0.142589,0.932136,0.136355,0.794411
6,FTI,8.50,N/A,0.917665,0.215302,0.864271,0.22598,0.952096,0.097666,0.858283,0.36533,0.996008
7,HRB,41.20,N/A,0.908184,0.67588,0.978044,0.706364,0.998004,0.593116,1.0,0.098403,0.656687
8,JKHY,217.54,N/A,0.905689,0.234044,0.872255,0.269054,0.976048,0.142118,0.928144,0.155063,0.846307
9,ANET,130.82,N/A,0.899202,0.344207,0.924152,0.036111,0.744511,0.144578,0.94012,0.320638,0.988024


### Calculating the Number of Shares to Buy

In [23]:
calculate_shares(hqm_stocks)

   Ticker  Stock Price No. of Shares to Buy HQM Score 1-Year Price Return  \
0     COG        23.16             863557.0   0.95509            0.353617   
1    SNPS       385.99              51814.0    0.9501            0.288718   
2    CDNS       187.93             106422.0    0.9501            0.237205   
3     AIV         8.65            2312138.0  0.939122            0.205268   
4     GWW       561.03              35648.0  0.929142            0.299526   
5     UNM        36.00             555555.0  0.919162            0.478379   
6     FTI         8.50            2352941.0  0.917665            0.215302   
7     HRB        41.20             485436.0  0.908184             0.67588   
8    JKHY       217.54              91937.0  0.905689            0.234044   
9    ANET       130.82             152881.0  0.899202            0.344207   
10     LW        83.40             239808.0  0.898703            0.292553   
11    ADP       248.60              80450.0  0.895709             0.18138   

c:\Python310\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## Formatting Our Excel Output

In [18]:
writer = pd.ExcelWriter("momentum_strategy.xlsx", engine="xlsxwriter")
hqm_stocks.to_excel(writer, "Momentum Strategy", index=False)

### Creating the Formats

In [19]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats

Sample Code:

```python
writer.sheets['Recommended Trades'].set_column('B:B', # apply the format to column B
                     18, # column width of 18 pixels
                     string_template # applies the previously-built format 'string_template' to the column
                     )
```

In [20]:
column_formats = {
    "A":["Ticker", string_template],
    "B":["Stock Price", dollar_template],
    "C":["No. of Shares to Buy", integer_template],
    "D":["HQM Score", percent_template],
    "E":["1-Year Price Return", percent_template],
    "F":["1-Year Return Percentile", percent_template],
    "G":["6-Month Price Return", percent_template],
    "H":["6-Month Return Percentile", percent_template],
    "I":["3-Month Price Return", percent_template],
    "J":["3-Month Return Percentile", percent_template],
    "K":["1-Month Price Return", percent_template],
    "L":["1-Month Return Percentile", percent_template]
}

for column in column_formats:
    writer.sheets["Momentum Strategy"].set_column(f"{column}:{column}", 25, column_formats[column][1])
    writer.sheets["Momentum Strategy"].write(f"{column}1", column_formats[column][0], column_formats[column][1])

In [21]:
writer.save()